In [1005]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import model_afolu as ma
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_electricity as ml
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import sqlalchemy
import sqlite3
from typing import Union
# very useful: https://stackoverflow.com/questions/427453/how-can-i-get-the-source-code-of-a-python-function
#print(inspect.getsource(FUNCTIONHERE))
import inspect
import sql_utilities as sqlutil
from julia.api import Julia

importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
#importlib.reload(ml)


<module 'model_socioeconomic' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_socioeconomic.py'>

In [909]:
sa.model_attributes.configuration.valid_solver

['cbc', 'clp', 'cplex', 'glpk', 'gurobi']

In [10]:
from julia import Base
from julia import Pkg
Pkg.activate(sa.dir_jl)

from julia import NemoMod
from julia import Cbc
from julia import JuMP



In [857]:
sa.model_attributes.configuration.get("nemo_mod_solver")

'clp'

In [515]:
NemoMod.createnemodb(sa.fp_sqlite_nemomod_db_tmp)

<PyCall.jlwrap SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite")>

In [260]:
isinstance(engine, sqlalchemy.engine.Engine)

True

In [121]:
engine = sqlalchemy.create_engine(f"sqlite:///{sa.fp_sqlite_nemomod_db_tmp}")

In [23]:
with engine.connect() as con:
    df1 = pd.read_sql_query("select * from OutputActivityRatio;", con)
#df1

In [46]:
sqlutil._write_dataframes_to_db(
    dict_to_sql2,
    engine
)

In [904]:
str(engine.url).replace("sqlite:///", "")

'/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite'

In [558]:
##  GENERATE INPUT DICTIONARY

importlib.reload(sf)
importlib.reload(ml)
model_elec = ml.ElectricEnergy(sa.model_attributes)
fp_in = sa.dict_fp_csv_nemomod.get(sa.model_attributes.table_nemomod_specified_demand_profile)
df_spd = pd.read_csv(fp_in)

fp_in = sa.dict_fp_csv_nemomod[sa.model_attributes.table_nemomod_capacity_factor]
df_capfactor = pd.read_csv(fp_in)

#dict_from_storage, dict_to_storage, dict_storage_techs_to_storage = model_elec.format_nemomod_table_technology_from_and_to_storage()
t0 =time.time()
dict_to_sql = model_elec.generate_input_tables_for_sql(df_cs_integrated, df_capfactor, df_spd);


##  REDUCED FOR TESTINS

dict_to_sql2 = {}
#dict_to_sql2["CapacityToActivityUnit"]
write_always = [
    "EMISSION", 
    "FUEL", 
    "MODE_OF_OPERATION",
    "REGION",
    "STORAGE",
    "TECHNOLOGY",
    "YEAR",
    "LTsGroup",
    "TIMESLICE",
    "TSGROUP1",
    "TSGROUP2",
    "YearSplit"
]
others = sorted([
    x for x in dict_to_sql.keys() if x not in write_always
])

years_keep = ["0", "1"]
for k in write_always + others:
    df_tmp = dict_to_sql[k].copy() 
    if "y" in df_tmp.columns:  
        dict_to_sql2[k] = df_tmp[df_tmp["y"].isin(years_keep)].reset_index(drop = True)
    elif k == "YEAR":
        dict_to_sql2[k] = df_tmp[df_tmp["val"].isin(years_keep)].reset_index(drop = True)
    else:
        dict_to_sql2[k] = df_tmp


sqlutil._write_dataframes_to_db(
    dict_to_sql,
    engine
)


In [905]:
base = 1000
optimizer = JuMP.Model(Cbc.Optimizer)
NemoMod.calculatescenario(
    sa.fp_sqlite_nemomod_db_tmp, 
    jumpmodel = optimizer, 
    numprocs = 1, 
    calcyears = [base, base + 10, base + 20, base + 30, base + 35],#[1, 36],#10, 35],
    reportzeros = False,
    varstosave = "vdiscountedcapitalinvestment, vtotalcapacityannual, vcapitalinvestment, vdiscountedcapitalinvestmentstorage, vaccumulatednewstoragecapacity, vproductionbytechnologyannual, vdiscountedoperatingcost, vannualtechnologyemission"
)


<PyCall.jlwrap OPTIMAL>

In [944]:
from julia import Clp

In [959]:
importlib.reload(sf)
importlib.reload(sqlutil)
importlib.reload(ml)

model_elec = ml.ElectricEnergy(sa.model_attributes, sa.dir_ref_nemo)


In [ ]:

t0 = time.time()
df_elec = model_elec.project(
    df_cs_integrated, 
    engine
    #vector_calc_time_periods = [0, 5, 10]
)
t1 = time.time()

In [961]:
#
# add fuel costs
# add natural gas minimum
# add waste minimum
#
"all".split(",")

['all']

In [766]:
df_proj[[x for x in df_proj.columns if ("entc" in x) and (("oil" in x) or ("natural_gas" in x) or ("coal" in x))]].tail()




,nemomod_entc_discounted_capital_investment_pp_coal,nemomod_entc_discounted_capital_investment_pp_oil,nemomod_entc_discounted_operating_costs_pp_coal,nemomod_entc_discounted_operating_costs_pp_oil,nemomod_entc_total_annual_generation_capacity_pp_coal,nemomod_entc_total_annual_generation_capacity_pp_oil,nemomod_entc_annual_production_by_technology_pp_coal,nemomod_entc_annual_production_by_technology_pp_oil
31,0.0,0.0,0.0,920.264237,0.0,32.0,0.0,36.794379
32,0.0,0.0,0.0,893.672369,0.0,32.0,0.0,37.413649
33,0.0,0.0,0.0,867.853149,0.0,32.0,0.0,38.045860
34,0.0,0.0,0.0,842.784291,0.0,32.0,0.0,38.691911
35,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [793]:
dict_to_sql["InputActivityRatio"][
    dict_to_sql["InputActivityRatio"]["f"].isin(["fuel_oil"])
]["val"].unique()



array([2.5])

In [786]:
dict_to_sql["OutputActivityRatio"][
    dict_to_sql["OutputActivityRatio"]["t"].isin(["supply_fuel_oil"])
].tail()

,id,r,t,f,m,y,val
787,788,costa_rica,supply_fuel_oil,fuel_oil,generate,1031,1
788,789,costa_rica,supply_fuel_oil,fuel_oil,generate,1032,1
789,790,costa_rica,supply_fuel_oil,fuel_oil,generate,1033,1
790,791,costa_rica,supply_fuel_oil,fuel_oil,generate,1034,1
791,792,costa_rica,supply_fuel_oil,fuel_oil,generate,1035,1


In [826]:
with engine.connect() as con:
    df1 = pd.read_sql_query("select * from vproductionbytechnologyannual;", con)
df1[df1["t"] == "pp_oil"]

,r,t,f,y,val,solvedtm
23,costa_rica,pp_oil,fuel_electricity,1030,36.187311,2022-08-17 19:15:33.162


In [804]:
with engine.connect() as con:
    df1 = pd.read_sql_query("select * from InputActivityRatio;", con)
#df1[df1["t"] == "pp_oil"]
df1[df1["f"] == "fuel_oil"]["t"].unique()

array(['pp_oil'], dtype=object)

In [821]:
for k in dict_to_sql.keys():
    if "t" in dict_to_sql[k].columns:
        if "pp_oil" in list(dict_to_sql[k]["t"]):
            print(k)

OperationalLife
CapitalCost
FixedCost
VariableCost
EmissionActivityRatio
InputActivityRatio
OutputActivityRatio
ReserveMarginTagTechnology
ResidualCapacity


In [818]:
"supply_fuel_oil" in dict_to_sql["InputActivityRatio"]["t"]

False

In [830]:
with engine.connect() as con:
    df1 = pd.read_sql_query("select * from InputActivityRatio;", con)
#df1[df1["t"] == "supply_fuel_oil"]
df1[df1["t"] == "pp_oil"]

,id,r,t,f,m,y,val
252,253,costa_rica,pp_oil,fuel_oil,generate,1000,2.5
253,254,costa_rica,pp_oil,fuel_oil,generate,1001,2.5
254,255,costa_rica,pp_oil,fuel_oil,generate,1002,2.5
255,256,costa_rica,pp_oil,fuel_oil,generate,1003,2.5
256,257,costa_rica,pp_oil,fuel_oil,generate,1004,2.5
257,258,costa_rica,pp_oil,fuel_oil,generate,1005,2.5
258,259,costa_rica,pp_oil,fuel_oil,generate,1006,2.5
259,260,costa_rica,pp_oil,fuel_oil,generate,1007,2.5
260,261,costa_rica,pp_oil,fuel_oil,generate,1008,2.5
261,262,costa_rica,pp_oil,fuel_oil,generate,1009,2.5


In [ ]:
model_elec.retrieve_nemomod_technology_table(
    engine,
    model_elec.modvar_enfu_energy_demand_by_fuel_elec,
    model_elec.model_attributes.table_nemomod_capital_investment_storage_discounted,
    df_cs_integrated["time_period"],
    techs_to_pivot = ["all_techs_dummy"]
)

In [839]:
model_elec.retrieve_nemomod_technology_table(
    engine,
    model_elec.modvar_enst_nemomod_discounted_capital_investment_storage,
    model_elec.model_attributes.table_nemomod_capital_investment_storage_discounted,
    df_cs_integrated["time_period"],                                
    field_pivot = model_elec.field_nemomod_storage,
    techs_to_pivot = ["all_techs_strg"]
)

,nemomod_enst_discounted_capital_investment_st_batteries,nemomod_enst_discounted_capital_investment_st_compressed_air,nemomod_enst_discounted_capital_investment_st_flywheels
0,0.0,0.000000,0.0
1,0.0,0.000000,0.0
2,0.0,0.000000,0.0
3,0.0,0.000000,0.0
4,0.0,0.000000,0.0
5,0.0,0.000000,0.0
6,0.0,0.000000,0.0
7,0.0,0.000000,0.0
8,0.0,0.000000,0.0
9,0.0,0.000000,0.0


In [836]:
model_elec.model_attributes.table_nemomod_capital_investment_storage_discounted

'vdiscountedcapitalinvestmentstorage'

In [840]:
with engine.connect() as con:
    df1 = pd.read_sql_query(f"select * from {model_elec.model_attributes.table_nemomod_capital_investment_storage_discounted};", con)
#df1[df1["e"] == "co2"]
df1#[df1["y"] == "2038"]#[df1["y"] == "2055"]#[df1["f"] == "fuel_electricity"]

,r,s,y,val,solvedtm
0,costa_rica,st_compressed_air,1035,7.237571,2022-08-17 19:31:05.709


In [750]:
with engine.connect() as con:
    df1 = pd.read_sql_query("select * from vproductionbytechnologyannual;", con)
#df1[df1["e"] == "co2"]
df1#[df1["y"] == "2038"]#[df1["y"] == "2055"]#[df1["f"] == "fuel_electricity"]

,r,t,f,y,val,solvedtm
0,costa_rica,st_compressed_air,fuel_electricity,1026,8.471239,2022-08-17 13:10:03.293
1,costa_rica,pp_solar,fuel_electricity,1004,4.539159,2022-08-17 13:10:03.293
2,costa_rica,st_flywheels,fuel_electricity,1016,7.195964,2022-08-17 13:10:03.293
3,costa_rica,supply_fuel_solar,fuel_solar,1004,19.850220,2022-08-17 13:10:03.293
4,costa_rica,supply_fuel_geothermal,fuel_geothermal,1011,19.060246,2022-08-17 13:10:03.293
...,...,...,...,...,...,...
196,costa_rica,st_batteries,fuel_electricity,1020,7.682156,2022-08-17 13:10:03.293
197,costa_rica,st_batteries,fuel_electricity,1018,7.435618,2022-08-17 13:10:03.293
198,costa_rica,pp_solar,fuel_electricity,1021,5.991026,2022-08-17 13:10:03.293
199,costa_rica,st_batteries,fuel_electricity,1016,7.195964,2022-08-17 13:10:03.293


In [977]:
importlib.reload(ds)
importlib.reload(mi)

warnings.filterwarnings("ignore")#ignore

df_cs_integrated = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))

model_afolu = ma.AFOLU(sa.model_attributes);
model_circecon = mc.CircularEconomy(sa.model_attributes);
model_ippu = mi.IPPU(sa.model_attributes);
model_energy = me.NonElectricEnergy(sa.model_attributes);

# 
df_project_afolu = model_afolu.project(df_cs_integrated)
df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_afolu], sa.model_attributes, "concatenate")

# pass integrated df
df_project_ce = model_circecon.project(df_cs_integrated)
df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_ce], sa.model_attributes, "concatenate")

# pass integrated df
df_project_ip = model_ippu.project(df_cs_integrated)
df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_ip], sa.model_attributes, "concatenate")

# pass integrated df
df_project_en = model_energy.project(df_cs_integrated)
df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_en], sa.model_attributes, "concatenate")


/Users/jsyme/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [1006]:
import sector_models as sm
importlib.reload(sm)

models_run = ["AFOLU", "CircularEconomy", "IPPU", "NonElectricEnergy"]
df_output_data = []
df_input_data = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))
run_integrated_q = True
# run AFOLU and collect output
if "AFOLU" in models_run:
    print("\n\tRunning AFOLU")
    # get the model, run it using the input data, then update the output data (for integration)
    model_afolu = sm.AFOLU(sa.model_attributes)
    df_output_data.append(model_afolu.project(df_input_data))

print(list(df_input_data.columns).count("qty_lvst_domestic_animal_mass_mt"))
print("\n")

# run CircularEconomy and collect output
if "CircularEconomy" in models_run:
    print("\n\tRunning CircularEconomy")
    model_circecon = sm.CircularEconomy(sa.model_attributes)
    # integrate AFOLU output?
    if run_integrated_q and set(["AFOLU"]).issubset(set(models_run)):
        df_input_data = sa.model_attributes.transfer_df_variables(
            df_input_data,
            df_output_data[0],
            model_circecon.integration_variables
        )
    df_output_data.append(model_circecon.project(df_input_data))
    df_output_data = [sf.merge_output_df_list(df_output_data, sa.model_attributes, "concatenate")] if run_integrated_q else df_output_data

print(list(df_input_data.columns).count("qty_lvst_domestic_animal_mass_mt"))
print("\n")

# run IPPU and collect output
if "IPPU" in models_run:
    print("\n\tRunning IPPU")
    model_ippu = sm.IPPU(sa.model_attributes)
    # integrate Circular Economy output?
    if run_integrated_q and set(["CircularEconomy"]).issubset(set(models_run)):
        df_input_data = sa.model_attributes.transfer_df_variables(
            df_input_data,
            df_output_data[0],
            model_ippu.integration_variables
        )
    df_output_data.append(model_ippu.project(df_input_data))
    df_output_data = [sf.merge_output_df_list(df_output_data, sa.model_attributes, "concatenate")] if run_integrated_q else df_output_data

print(list(df_input_data.columns).count("qty_lvst_domestic_animal_mass_mt"))
print("\n")

# run Non-Electric Energy and collect output
if "NonElectricEnergy" in models_run:
    print("\n\tRunning NonElectricEnergy")
    model_energy = sm.NonElectricEnergy(sa.model_attributes)
    # integrate IPPU output?
    if run_integrated_q and set(["IPPU", "AFOLU"]).issubset(set(models_run)):
        df_input_data = sa.model_attributes.transfer_df_variables(
            df_input_data,
            df_output_data[0],
            model_energy.integration_variables
        )
        print(list(df_input_data.columns).count("qty_lvst_domestic_animal_mass_mt"))
        print("\n")
    else:
        print("LOG ERROR HERE: CANNOT RUN WITHOUT IPPU")
    df_output_data.append(model_energy.project(df_input_data))
    df_output_data = [sf.merge_output_df_list(df_output_data, sa.model_attributes, "concatenate")] if run_integrated_q else df_output_data



	Running AFOLU
0



	Running CircularEconomy
1



	Running IPPU
1



	Running NonElectricEnergy
1




In [996]:
sa.model_attributes.get_standard_variables(
    df_input_data, 
    model_afolu.modvar_lvst_total_animal_mass
)

ValueError: Shape of passed values is (36, 4), indices imply (36, 1)

In [126]:
julia.install()

,id,r,t,y,val
